In [1]:
import sys
sys.path.append('/home/heyangle/Desktop/ScanCraft/ScanCraft')
sys.path.append('C:\\Users\\vooum\\Source\\Repos\\ScanCraft\\ScanCraft')
from command.scan import scan
from command.NMSSMTools import NMSSMTools
mold=scan(method='random')
mold.AddScalar('tanB','MINPAR',3,1.,60.)
mold.AddScalar('M1','EXTPAR',1  ,20.    ,1000.)
mold.AddScalar('M2','EXTPAR'   ,2  ,100.    ,2000.)
mold.AddScalar('Atop','EXTPAR'   ,11  ,  -6e3    ,6e3)
mold.AddFollower('Abottom','EXTPAR'   ,12,'Atop')
mold.AddScalar('Atau','EXTPAR'   ,13  ,  100.      ,2000.)
mold.AddFollower('MtauL','EXTPAR'   ,33,'Atau')
mold.AddFollower('MtauR','EXTPAR'   ,36,'Atau')
mold.AddScalar('MQ3L','EXTPAR'   ,43,	100.,	2.e3)
mold.AddScalar('MtopR'	,'EXTPAR'   ,46,	100.,	2.e3)
mold.AddFollower('MbottomR','EXTPAR'  ,49,'MtopR')
mold.AddScalar('Lambda','EXTPAR'  ,61  ,1e-3    ,1. ,prior_distribution='exponential')
mold.AddScalar('Kappa','EXTPAR'   ,62 ,1.e-3    ,1. ,prior_distribution='exponential')
mold.AddScalar('A_Lambda','EXTPAR' ,63,-3.e3,3.e3)
mold.AddScalar('A_kappa','EXTPAR' ,64,-3.e3,3.e3)
mold.AddScalar('mu_eff','EXTPAR'  ,65,100.,1500.)

In [2]:
from command.multi_thread.queue_operation import GenerateQueue,FillQueue
from command.multi_thread.MT_NTools import MT_NTools
MTN=MT_NTools(threads=6)

delete folder record? (y/n) 



In [3]:
ore_q=GenerateQueue(mold,lenth=300)
# ore_q.qsize()
MTN.run(ore_q)

Calculations begin at Fri Dec  8 22:22:25 2017
  threads:	6
  points:	300
  thread-2	runing,      200 points left at Fri Dec  8 22:22:31 2017
  thread-1	runing,      100 points left at Fri Dec  8 22:22:37 2017
  thread-0	runing,        0 points left at Fri Dec  8 22:22:50 2017
All points done. Use 0.010959 hours
43 sample recorded in ./Pylon/record


In [11]:
# from command.data_transformer.ArrayToPoint import ArrayToInputQueue
# from command.format.data_structure_functions import FlatToList
# import numpy
# q=ArrayToInputQueue(numpy.random.randn(2,12),mold)

In [4]:
acc=MTN.accepted_list
exc=MTN.excluded_list
from command.data_transformer.InputListToPandas import InputListToPandas as I2P
DF_acc=I2P(acc)
DF_exc=I2P(exc)
last_name=('input','points','calculable','1=y,0=n')
DF_acc[last_name]=1
DF_exc[last_name]=0

In [5]:
import numpy
data=numpy.concatenate([DF_acc.values,DF_exc.values],axis=0)

In [6]:
numpy.save('data3',data)

In [70]:
import torch
from torch.autograd import Variable
from torch import nn
bce=nn.BCEWithLogitsLoss()
in_data=Variable(torch.FloatTensor(data[:,:-1]).cuda())
target=Variable(torch.FloatTensor(data[:,-1].reshape(300,1)).cuda())

In [71]:
model=torch.nn.Sequential(
    torch.nn.Linear(12,1000),
    torch.nn.Linear(1000,10),
    torch.nn.Linear(10,1),
#     torch.nn.ReLU(),
    torch.nn.Sigmoid()
).cuda()

In [72]:
model

Sequential (
  (0): Linear (12 -> 1000)
  (1): Linear (1000 -> 10)
  (2): Linear (10 -> 1)
  (3): Sigmoid ()
)

In [73]:
for param in model.parameters():
    print(param)

Parameter containing:
-0.1254 -0.0615  0.0552  ...  -0.0587 -0.2114  0.1890
 0.2278 -0.0013  0.1871  ...  -0.0116 -0.2759 -0.0507
 0.0381  0.0951 -0.0544  ...  -0.2746  0.0301  0.1539
          ...             ⋱             ...          
 0.2527  0.2605 -0.1895  ...   0.2237 -0.2564  0.2719
 0.1371 -0.1007 -0.1792  ...   0.0274  0.1545  0.0189
 0.1718  0.2471 -0.2370  ...   0.1389  0.1520  0.1220
[torch.cuda.FloatTensor of size 1000x12 (GPU 0)]

Parameter containing:
 0.1203
 0.0593
 0.2002
   ⋮   
-0.1752
-0.2745
-0.0191
[torch.cuda.FloatTensor of size 1000 (GPU 0)]

Parameter containing:
 3.4886e-03 -1.7333e-02 -1.6789e-02  ...   1.8569e-04 -1.4737e-02  1.7576e-02
 3.0980e-02 -1.4178e-02 -4.4440e-03  ...  -1.3244e-02 -1.7774e-03 -9.7443e-03
-1.7543e-02 -2.0296e-02 -1.6417e-02  ...  -1.4773e-02 -6.0899e-03 -3.1296e-03
                ...                   ⋱                   ...                
 2.7893e-02 -2.2324e-02  3.1146e-02  ...   1.4862e-02  7.7109e-03  1.9427e-02
 1.4459e-02  

In [74]:
model(in_data)

Variable containing:
 1.0000
 1.0000
 0.0000
 0.0000
 0.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 0.0000
 1.0000
 0.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 0.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 0.0000
 1.0000
 0.0000
 0.0000
 1.0000
 0.0000
 1.0000
 0.0000
 0.0000
 1.0000
 0.0000
 0.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 0.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 0.0000
 0.0000
 0.0000
 1.0000
 0.0000
 1.0000
 1.0000
 1.0000
 0.0000
 0.0000
 1.0000
 0.0000
 0.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 0.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 0.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 0.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 0.0000
 0.0000
 1.0000
 0.0000
 1.0000
 1.0000
 0.0000
 0.0000
 0.0000
 1.0000
 1.0000
 0.0000
 1.0000
 0.0000
 1.0000
 0.2719
 1.0000
 1.0000
 1.0000
 0.0000
 1.0000
 1.0000
 1.0000
 0.0000
 1.0000
 1.0000
 1.

In [75]:
bce(_,target)

Variable containing:
 1.0133
[torch.cuda.FloatTensor of size 1 (GPU 0)]

In [77]:
for i in range(5000):
    out_data=model(in_data)    
    loss=bce(out_data,target)
    loss.backward()
    if i%100==0:print(i,loss)
    for param in model.parameters():
        param.data-=0.01*param.grad.data
    model.zero_grad()

0 Variable containing:
 1.0133
[torch.cuda.FloatTensor of size 1 (GPU 0)]

100 Variable containing:
 0.6931
[torch.cuda.FloatTensor of size 1 (GPU 0)]

200 Variable containing:
 0.6931
[torch.cuda.FloatTensor of size 1 (GPU 0)]

300 Variable containing:
 0.6931
[torch.cuda.FloatTensor of size 1 (GPU 0)]

400 Variable containing:
 0.6931
[torch.cuda.FloatTensor of size 1 (GPU 0)]

500 Variable containing:
 0.6931
[torch.cuda.FloatTensor of size 1 (GPU 0)]

600 Variable containing:
 0.6931
[torch.cuda.FloatTensor of size 1 (GPU 0)]

700 Variable containing:
 0.6931
[torch.cuda.FloatTensor of size 1 (GPU 0)]

800 Variable containing:
 0.6931
[torch.cuda.FloatTensor of size 1 (GPU 0)]

900 Variable containing:
 0.6931
[torch.cuda.FloatTensor of size 1 (GPU 0)]

1000 Variable containing:
 0.6931
[torch.cuda.FloatTensor of size 1 (GPU 0)]

1100 Variable containing:
 0.6931
[torch.cuda.FloatTensor of size 1 (GPU 0)]

1200 Variable containing:
 0.6931
[torch.cuda.FloatTensor of size 1 (GPU 0)]

In [78]:
out_data

Variable containing:
    0
    0
    0
    0
    0
    0
    0
    0
    0
    0
    0
    0
    0
    0
    0
    0
    0
    0
    0
    0
    0
    0
    0
    0
    0
    0
    0
    0
    0
    0
    0
    0
    0
    0
    0
    0
    0
    0
    0
    0
    0
    0
    0
    0
    0
    0
    0
    0
    0
    0
    0
    0
    0
    0
    0
    0
    0
    0
    0
    0
    0
    0
    0
    0
    0
    0
    0
    0
    0
    0
    0
    0
    0
    0
    0
    0
    0
    0
    0
    0
    0
    0
    0
    0
    0
    0
    0
    0
    0
    0
    0
    0
    0
    0
    0
    0
    0
    0
    0
    0
    0
    0
    0
    0
    0
    0
    0
    0
    0
    0
    0
    0
    0
    0
    0
    0
    0
    0
    0
    0
    0
    0
    0
    0
    0
    0
    0
    0
    0
    0
    0
    0
    0
    0
    0
    0
    0
    0
    0
    0
    0
    0
    0
    0
    0
    0
    0
    0
    0
    0
    0
    0
    0
    0
    0
    0
    0
    0
    0
    0
    0
    0
    0
 

In [17]:
target

Variable containing:
    1
    1
    1
    1
    1
    1
    1
    1
    1
    1
    1
    1
    1
    1
    1
    1
    1
    1
    1
    1
    1
    1
    1
    1
    1
    1
    1
    1
    1
    1
    1
    1
    1
    1
    1
    1
    1
    1
    0
    0
    0
    0
    0
    0
    0
    0
    0
    0
    0
    0
    0
    0
    0
    0
    0
    0
    0
    0
    0
    0
    0
    0
    0
    0
    0
    0
    0
    0
    0
    0
    0
    0
    0
    0
    0
    0
    0
    0
    0
    0
    0
    0
    0
    0
    0
    0
    0
    0
    0
    0
    0
    0
    0
    0
    0
    0
    0
    0
    0
    0
    0
    0
    0
    0
    0
    0
    0
    0
    0
    0
    0
    0
    0
    0
    0
    0
    0
    0
    0
    0
    0
    0
    0
    0
    0
    0
    0
    0
    0
    0
    0
    0
    0
    0
    0
    0
    0
    0
    0
    0
    0
    0
    0
    0
    0
    0
    0
    0
    0
    0
    0
    0
    0
    0
    0
    0
    0
    0
    0
    0
    0
    0
    0
 